In [1]:
%load_ext autoreload
%autoreload 2

# Import dependencies


In [2]:
import os

from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, BaseMessage
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import chain
from langgraph.prebuilt import create_react_agent
from langchain_core.utils.function_calling import tool_example_to_messages
from pydantic import BaseModel, Field
from langchain_community.tools import DuckDuckGoSearchRun
from enum import Enum
from langgraph.graph.message import add_messages
from langchain_core.messages import trim_messages
from langchain_google_genai import ChatGoogleGenerativeAI


from IPython.display import display, Markdown, Latex

from typing_extensions import Annotated, TypedDict
from typing import List, Optional, Sequence

/home/cuongdm/git-cuongpiger/langchain-labs/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Configure Google credentials

- **NOTE**: Remember change the `GOOGLE_APPLICATION_CREDENTIALS` to the path of your own Google credentials file.


In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "/home/cuongdm/git-cuongpiger/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
)

# Create the agents

In [4]:
# Create the agent
memory = MemorySaver()
model = ChatVertexAI(temperature=0, model="gemini-1.5-flash")
search = DuckDuckGoSearchRun(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use agents

In [5]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {
        "messages": [
            HumanMessage(
                content="Xin chào, hiện tại tôi đang sống ở thành phố Hồ Chí Minh"
            )
        ]
    },
    config,
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Xin chào! Thành phố Hồ Chí Minh là một thành phố tuyệt vời. Bạn thích điều gì nhất ở đây? 😊 \n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.03410044312477112, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.05261888727545738}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.0259573832154274, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.09009324759244919}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.07263684272766113, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.05500539019703865}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.05921032652258873,

In [6]:
for chunk in agent_executor.stream(
    {
        "messages": [
            HumanMessage(
                content="Hiện tại thời tiết ở thành phố Hồ Chí Minh đang như thế nào?"
            )
        ]
    },
    config,
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Rất tiếc, tôi không thể cung cấp thông tin thời tiết hiện tại.  Bạn có thể thử tìm kiếm trên Google hoặc xem dự báo thời tiết trên điện thoại của bạn. 😊 \n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.020964214578270912, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.06187598779797554}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.04813668131828308, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.04535272344946861}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.1285252720117569, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.05033065751194954}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_l

# Define tools

In [7]:
search = DuckDuckGoSearchRun(max_results=2)
search_results = search.invoke("Thời tiết thành phố Hồ Chí Minh như thế nào?")

In [8]:
search_results

'Thời Tiết TP.HCM cập nhật diễn biến thời tiết tại khắp các quận, huyện Thành phố Hồ Chí Minh nhanh chóng, chính xác và chi tiết nhất. Ngoài ra, trang web tổng hợp và chia sẻ những thông tin hữu ích về thời tiết, du lịch, ẩm thực, và văn hóa ở TP.HCM. Web site nchmf.gov.vn - web site thông tin dự báo thời tiết biển, đất liền 24h,48h,72h, sản phẩm mô hình, ảnh Radar, ảnh vệ tinh. Giới thiệu. ... Thời tiết Hồ Chí Minh. Thời tiết hiện tại. Cập nhật: 1h 20/01/2025. Dự báo thời tiết ngày mai tại Hồ Chí Minh. Cập nhật thông tin nhiệt độ, độ ẩm, lượng mưa chính xác và chi tiết nhất tại khu vực Hồ Chí Minh ... Tỉnh - Thành phố Đông Bắc Bộ ... Tác hại của tia Uv như thế nào đối với chúng ta hay chưa. Tình hình dự báo thời tiết, nhiệt độ ở Hồ Chí Minh hôm nay, ngày mai và các ngày tới. ... Giờ địa phương: Tỉnh/Thành phố. ... Dự báo thời tiết Hồ Chí Minh những ngày tới. Ngày. Đêm. Chủ nhật 19/01. mây thưa 4.95 km/h. 24° / 35° ... Dự báo thời tiết 10 ngày tới tại Hồ Chí Minh. Thông tin mười ngày t

In [9]:
tools = [search]

In [10]:
model = ChatVertexAI(temperature=0, model="gemini-1.5-flash")

In [11]:
model_with_tools = model.bind_tools(tools)

In [12]:
response = model_with_tools.invoke(
    [HumanMessage(content="Xin chào, hiện tại tôi đang sống ở thành phố Hồ Chí Minh")]
)

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Xin chào! Thành phố Hồ Chí Minh là một thành phố tuyệt vời. Bạn thích điều gì nhất ở đây? 😊 

ToolCalls: []


In [13]:
response = model_with_tools.invoke(
    [HumanMessage(content="Thời tiết của thành phố Hồ Chí Minh hiện tại như thế nào?")]
)

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'duckduckgo_search', 'args': {'query': 'weather in Ho Chi Minh City'}, 'id': 'd2b240c1-87b1-4ab6-990a-24fed1722a8c', 'type': 'tool_call'}]


In [20]:
agent_executor = create_react_agent(model, tools)

In [21]:
response = agent_executor.invoke(
    {
        "messages": [
            HumanMessage(
                content="Thời tiết của thành phố Hồ Chí Minh hiện tại như thế nào?"
            )
        ]
    }
)

In [16]:
response["messages"]

[HumanMessage(content='Thời tiết của thành phố Hồ Chí Minh hiện tại như thế nào?', additional_kwargs={}, response_metadata={}, id='ed45d351-ae0b-45f0-9b99-7ee1bd3ded07'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'duckduckgo_search', 'arguments': '{"query": "weather in Ho Chi Minh City"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.07470703125, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.09033203125}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.130859375, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.1630859375}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.103515625, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.08251953125}, {'categ

In [22]:
for chunk in agent_executor.stream(
    {
        "messages": [
            HumanMessage(
                content="Thời tiết của thành phố Hồ Chí Minh hiện tại như thế nào?"
            )
        ]
    }
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'duckduckgo_search', 'arguments': '{"query": "weather in Ho Chi Minh City"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.07470703125, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.09033203125}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.130859375, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.1630859375}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.103515625, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.08251953125}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.046630859375, 'blocked': False, 'severity': 'HARM

# Streaming tokens

In [23]:
async for event in agent_executor.astream_events(
    {
        "messages": [
            HumanMessage(
                content="Thời tiết của thành phố Hồ Chí Minh hiện tại như thế nào?"
            )
        ]
    },
    version="v1",
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: duckduckgo_search with inputs: {'query': 'weather in Ho Chi Minh City'}
Done tool: duckduckgo_search
Tool output was: content="Today, in Ho Chi Minh City, cloudy weather is anticipated. The temperature will be between the lowest temperature of 22°C (71.6°F) and the highest temperature of 32°C (89.6°F). The hottest part of the day will be at 14h and 15h. Get the current weather conditions and 7-day forecast for Ho Chi Minh City, Vietnam. Learn about the tropical wet and dry climate, monthly averages, and historical weather data. See the links below the 12-day Ho Chi Minh City weather forecast table for other cities and towns nearby along with weather conditions for local outdoor activities. Ho Chi Minh City is 11 m above sea level and located at 10.77° N 106.72° E. Ho Chi Minh City has a population of 3467331. Local time in Ho Chi Minh City is +07. Ho Chi Minh City Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, map

# Adding in memory

In [24]:
memory = MemorySaver()

In [25]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [26]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {
        "messages": [
            HumanMessage(content="Thời tiết của thành phố Hồ Chí Minh hôm nay ra sao?")
        ]
    },
    config,
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'duckduckgo_search', 'arguments': '{"query": "weather in Ho Chi Minh City"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.083984375, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.09130859375}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.12353515625, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.1630859375}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.103515625, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.083984375}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.04736328125, 'blocked': False, 'severity': 'HARM_SE

In [27]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Tôi vừa hỏi gì vậy?")]},
    config,
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Bạn vừa hỏi về thời tiết của thành phố Hồ Chí Minh hôm nay. \n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.023330826312303543, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.09807942807674408}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.041462209075689316, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.09534948319196701}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.09807942807674408, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.07263684272766113}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.05921032652258873, 'blocked': False, 'severity'

In [28]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Thời tiết ở Ninh Bình như thế nào?")]},
    config,
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'duckduckgo_search', 'arguments': '{"query": "weather in Ninh Binh"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.0888671875, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.1025390625}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.1513671875, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.185546875}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.103515625, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.083984375}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.059326171875, 'blocked': False, 'severity': 'HARM_SEVERITY_N

In [29]:
for chunk in agent_executor.stream(
    {
        "messages": [
            HumanMessage(content="Thời tiết ở Ninh Bình trong 3 ngày tiếp theo ra sao?")
        ]
    },
    config,
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'duckduckgo_search', 'arguments': '{"query": "weather in Ninh Binh for the next 3 days"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.0791015625, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.1025390625}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.111328125, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.1416015625}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.111328125, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.08740234375}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.057373046875, 'blocked': False, 'sever

In [30]:
for chunk in agent_executor.stream(
    {
        "messages": [
            HumanMessage(
                content="Cần lưu ý gì nếu đi du lịch ở Ninh Bình vào thời gian này?"
            )
        ]
    },
    config,
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Thời tiết ở Ninh Bình hiện tại khá mát mẻ, phù hợp cho việc du lịch. Tuy nhiên, bạn nên lưu ý một số điều sau:\n\n* **Chuẩn bị áo ấm:**  Nhiệt độ có thể xuống thấp vào buổi tối và sáng sớm, nên bạn cần mang theo áo khoác hoặc áo len để giữ ấm.\n* **Mang theo ô hoặc áo mưa:**  Có thể có mưa vào một số ngày, nên bạn cần chuẩn bị ô hoặc áo mưa để tránh bị ướt.\n* **Bảo vệ da:**  Mặc dù thời tiết mát mẻ, nhưng ánh nắng mặt trời vẫn có thể gây hại cho da. Bạn nên thoa kem chống nắng và đội mũ nón khi ra ngoài.\n* **Chuẩn bị giày dép phù hợp:**  Ninh Bình có nhiều địa điểm du lịch đòi hỏi phải đi bộ nhiều, nên bạn cần mang theo giày dép thoải mái và phù hợp với địa hình.\n\nNgoài ra, bạn cũng nên tìm hiểu thêm về các điểm du lịch, ẩm thực và văn hóa của Ninh Bình để có một chuyến du lịch trọn vẹn. \n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGL